In [1]:
import os
import sys
import FlowCytometryTools
from FlowCytometryTools import FCMeasurement
from FlowCytometryTools import ThresholdGate, PolyGate, QuadGate
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import glob
from matplotlib import cm
import warnings
from pylab import *

# Log10 treatment for selected channels of FACS raw data
def custom_log(original_sample,channelLst):
    # Copy the original sample
    new_sample = original_sample.copy()
    new_data = new_sample.data
    original_data = original_sample.data
    
    # Our transformation goes here
    for channel in channelLst:
        new_data[channel] = np.log10(original_data[channel])
    new_data = new_data.replace([np.inf, -np.inf], np.nan).dropna() # Removes all NaN entries
    new_sample.data = new_data
    return new_sample

# Recalculate the Ratio = FITC-A/PE-Texas Red-A
def ratio_recal(original_sample):
    # Copy the original sample
    new_sample = original_sample.copy()
    new_data = new_sample.data
    original_data = original_sample.data
    
    # Our recalculation goes here
    new_data['Ratio: FITC-A/PE-Texas Red-A'] = original_data['FITC-A']/original_data['PE-Texas Red-A']
    new_data = new_data.dropna() # Removes all NaN entries
    new_sample.data = new_data
    return new_sample

In [2]:
libs = ['Lib_0A','Lib_0B','Lib_1A','Lib_1B','Lib_2A','Lib_2B',\
        'Lib_3A','Lib_3B','Lib_5A','Lib_5B','Lib_8A','Lib_8B']
gates = ['P2','P3','P4','P5','P6','P7','P8','P9']
fcmeasure = {}
for lib in libs:
    for gate in gates:
        file = glob.glob('*'+lib+'_'+gate+'*')[0]
        print(file)
        fcmeasure[lib+'_'+gate] = FCMeasurement(ID = '', datafile = file)
rec_fcmeasure = {}
log_fcmeasure = {}
for lib in libs:
    for gate in gates:
        rec_fcmeasure[lib+'_'+gate] = ratio_recal(fcmeasure[lib+'_'+gate])
        log_fcmeasure[lib+'_'+gate] = custom_log(rec_fcmeasure[lib+'_'+gate], ['PE-Texas Red-A', 'FITC-A',\
                                            'Ratio: FITC-A/PE-Texas Red-A'])

Specimen_001_Sorting_Lib_0A_P2_001.fcs
Specimen_001_Sorting_Lib_0A_P3_001.fcs
Specimen_001_Sorting_Lib_0A_P4_001.fcs
Specimen_001_Sorting_Lib_0A_P5_001.fcs
Specimen_001_Sorting_Lib_0A_P6_001.fcs
Specimen_001_Sorting_Lib_0A_P7_001.fcs
Specimen_001_Sorting_Lib_0A_P8_001.fcs
Specimen_001_Sorting_Lib_0A_P9_001.fcs
Specimen_001_Sorting_Lib_0B_P2_002.fcs
Specimen_001_Sorting_Lib_0B_P3_002.fcs
Specimen_001_Sorting_Lib_0B_P4_002.fcs
Specimen_001_Sorting_Lib_0B_P5_002.fcs
Specimen_001_Sorting_Lib_0B_P6_002.fcs
Specimen_001_Sorting_Lib_0B_P7_002.fcs
Specimen_001_Sorting_Lib_0B_P8_002.fcs
Specimen_001_Sorting_Lib_0B_P9_002.fcs
Specimen_001_Sorting_Lib_1A_P2_003.fcs
Specimen_001_Sorting_Lib_1A_P3_003.fcs
Specimen_001_Sorting_Lib_1A_P4_003.fcs
Specimen_001_Sorting_Lib_1A_P5_003.fcs
Specimen_001_Sorting_Lib_1A_P6_003.fcs
Specimen_001_Sorting_Lib_1A_P7_003.fcs
Specimen_001_Sorting_Lib_1A_P8_003.fcs
Specimen_001_Sorting_Lib_1A_P9_003.fcs
Specimen_001_Sorting_Lib_1B_P2_004.fcs
Specimen_001_Sorting_Lib_

In [3]:
# /////////////////////////////////////////////////////////////
# get the overlap between each two neighbour bins on the Log10u space, using which to calculate the Log10u boundary of two neighbour bins
# The function to define the overlap between each two neighbour bins on the Log10u space
# Briefly, feed two arrays (Log10u of two bins) and a grid bin space; search the grid space one bin by another, extract subarray within this small bin from two arrays and compare with each other. When both arrays have members within this bin and the comtamination (ratio of member belonging to two arrays) > 1%, extract the minority append it into a new array. calculate the mean of the new array.
def find_ovarlap(array1,array2,bins):
    array1copy=array1
    array2copy=array2
    array1copy.sort()
    array2copy.sort()
    bins.sort()
    myarray=np.array([])
    for i,item in enumerate(bins):
        if i<len(bins)-1:
            thisbin=[bins[i],bins[i+1]]
            array1here=array1copy[(array1copy>thisbin[0]) & (array1copy<thisbin[1])]
            array2here=array2copy[(array2copy>thisbin[0]) & (array2copy<thisbin[1])]
            if len(array1here)>0 and len(array2here)>0:
                if len(array1here)<len(array2here) and float(len(array1here))/float(len(array2here))>0.01:
                    myarray=np.append(array1here,myarray)
                elif len(array2here)<len(array1here) and float(len(array2here))/float(len(array1here))>0.01:
                    myarray=np.append(array2here,myarray)
    return myarray

In [4]:
bins=np.linspace(-3.0, 3.0, 600)
from scipy.stats import norm
P2P3,P3P4,P4P5,P5P6,P6P7,P7P8,P8P9,P2_95,P9_5 = {},{},{},{},{},{},{},{},{}
for lib in libs:
    P2P3overlap = find_ovarlap(np.array(log_fcmeasure[lib+'_'+'P2'].data['Ratio: FITC-A/PE-Texas Red-A']), \
                         np.array(log_fcmeasure[lib+'_'+'P3'].data['Ratio: FITC-A/PE-Texas Red-A']), bins)
    P3P4overlap = find_ovarlap(np.array(log_fcmeasure[lib+'_'+'P3'].data['Ratio: FITC-A/PE-Texas Red-A']), \
                         np.array(log_fcmeasure[lib+'_'+'P4'].data['Ratio: FITC-A/PE-Texas Red-A']), bins)
    P4P5overlap = find_ovarlap(np.array(log_fcmeasure[lib+'_'+'P4'].data['Ratio: FITC-A/PE-Texas Red-A']), \
                         np.array(log_fcmeasure[lib+'_'+'P5'].data['Ratio: FITC-A/PE-Texas Red-A']), bins)
    P5P6overlap = find_ovarlap(np.array(log_fcmeasure[lib+'_'+'P5'].data['Ratio: FITC-A/PE-Texas Red-A']), \
                         np.array(log_fcmeasure[lib+'_'+'P6'].data['Ratio: FITC-A/PE-Texas Red-A']), bins)
    P6P7overlap = find_ovarlap(np.array(log_fcmeasure[lib+'_'+'P6'].data['Ratio: FITC-A/PE-Texas Red-A']), \
                         np.array(log_fcmeasure[lib+'_'+'P7'].data['Ratio: FITC-A/PE-Texas Red-A']), bins)
    P7P8overlap = find_ovarlap(np.array(log_fcmeasure[lib+'_'+'P7'].data['Ratio: FITC-A/PE-Texas Red-A']), \
                         np.array(log_fcmeasure[lib+'_'+'P8'].data['Ratio: FITC-A/PE-Texas Red-A']), bins)
    P8P9overlap = find_ovarlap(np.array(log_fcmeasure[lib+'_'+'P8'].data['Ratio: FITC-A/PE-Texas Red-A']), \
                         np.array(log_fcmeasure[lib+'_'+'P9'].data['Ratio: FITC-A/PE-Texas Red-A']), bins)
    P2P3[lib] = np.mean(P2P3overlap)
    P3P4[lib] = np.mean(P3P4overlap)
    P4P5[lib] = np.mean(P4P5overlap)
    P5P6[lib] = np.mean(P5P6overlap)
    P6P7[lib] = np.mean(P6P7overlap)
    P7P8[lib] = np.mean(P7P8overlap)
    P8P9[lib] = np.mean(P8P9overlap)
    P2_95[lib] = np.percentile(np.array(log_fcmeasure[lib+'_'+'P2'].data['Ratio: FITC-A/PE-Texas Red-A']),95)
    P9_5[lib] = np.percentile(np.array(log_fcmeasure[lib+'_'+'P9'].data['Ratio: FITC-A/PE-Texas Red-A']),5)
    print(lib)
    print('-----------------------')
    print('P2P3 overlap: ')
    print(len(P2P3overlap)) 
    print('P2P3 mean') 
    print(np.mean(P2P3overlap)) 
    print(norm.fit(P2P3overlap)) 
    print('') 
    print('P3P4 overlap: ')
    print(len(P3P4overlap)) 
    print('P3P4 mean') 
    print(np.mean(P3P4overlap)) 
    print(norm.fit(P3P4overlap)) 
    print('') 
    print('P4P5 overlap: ')
    print(len(P3P4overlap)) 
    print('P4P5 mean') 
    print(np.mean(P4P5overlap)) 
    print(norm.fit(P4P5overlap)) 
    print('') 
    print('P5P6 overlap: ')
    print(len(P5P6overlap)) 
    print('P5P6 mean') 
    print(np.mean(P5P6overlap)) 
    print(norm.fit(P5P6overlap)) 
    print('') 
    print('P6P7 overlap: ')
    print(len(P6P7overlap)) 
    print('P6P7 mean') 
    print(np.mean(P6P7overlap)) 
    print(norm.fit(P6P7overlap)) 
    print('') 
    print('P7P8 overlap: ')
    print(len(P7P8overlap)) 
    print('P7P8 mean') 
    print(np.mean(P7P8overlap)) 
    print(norm.fit(P7P8overlap)) 
    print('') 
    print('P8P9 overlap: ')
    print(len(P8P9overlap)) 
    print('P8P9 mean') 
    print(np.mean(P8P9overlap)) 
    print(norm.fit(P8P9overlap)) 
    print('') 

Lib_0A
-----------------------
P2P3 overlap: 
2619
P2P3 mean
0.6739606873458376
(0.6739606873458376, 0.01864648220620731)

P3P4 overlap: 
7102
P3P4 mean
0.3606683212673117
(0.3606683212673117, 0.016877488768000506)

P4P5 overlap: 
7102
P4P5 mean
0.048551360677176976
(0.048551360677176976, 0.018773816638789798)

P5P6 overlap: 
8253
P5P6 mean
-0.26610919566143215
(-0.26610919566143215, 0.018825470597071885)

P6P7 overlap: 
6902
P6P7 mean
-0.5791589137921296
(-0.5791589137921296, 0.019558725538680512)

P7P8 overlap: 
7947
P7P8 mean
-0.8900105269041934
(-0.8900105269041934, 0.01743536450478033)

P8P9 overlap: 
20073
P8P9 mean
-1.2097319505475166
(-1.2097319505475166, 0.014509155651524233)

Lib_0B
-----------------------
P2P3 overlap: 
2541
P2P3 mean
0.6745626733135117
(0.6745626733135117, 0.018315307957980153)

P3P4 overlap: 
7311
P3P4 mean
0.3621151863330204
(0.3621151863330204, 0.016958650035479177)

P4P5 overlap: 
7311
P4P5 mean
0.04969423530040255
(0.04969423530040255, 0.01904753249711

In [5]:
boundary = {}
for lib in libs:
    boundary[lib] = {}
    boundary[lib]['Bin1'] = '%s,%s'%(round(P2P3[lib],5),round(P2_95[lib],5))
    boundary[lib]['Bin2'] = '%s,%s'%(round(P3P4[lib],5),round(P2P3[lib],5))
    boundary[lib]['Bin3'] = '%s,%s'%(round(P4P5[lib],5),round(P3P4[lib],5))
    boundary[lib]['Bin4'] = '%s,%s'%(round(P5P6[lib],5),round(P4P5[lib],5))
    boundary[lib]['Bin5'] = '%s,%s'%(round(P6P7[lib],5),round(P5P6[lib],5))
    boundary[lib]['Bin6'] = '%s,%s'%(round(P7P8[lib],5),round(P6P7[lib],5))
    boundary[lib]['Bin7'] = '%s,%s'%(round(P8P9[lib],5),round(P7P8[lib],5))
    boundary[lib]['Bin8'] = '%s,%s'%(round(P9_5[lib],5),round(P8P9[lib],5))

In [6]:
boundary

{'Lib_0A': {'Bin1': '0.67396,0.92539',
  'Bin2': '0.36067,0.67396',
  'Bin3': '0.04855,0.36067',
  'Bin4': '-0.26611,0.04855',
  'Bin5': '-0.57916,-0.26611',
  'Bin6': '-0.89001,-0.57916',
  'Bin7': '-1.20973,-0.89001',
  'Bin8': '-1.78343,-1.20973'},
 'Lib_0B': {'Bin1': '0.67456,0.93268',
  'Bin2': '0.36212,0.67456',
  'Bin3': '0.04969,0.36212',
  'Bin4': '-0.26546,0.04969',
  'Bin5': '-0.57726,-0.26546',
  'Bin6': '-0.88952,-0.57726',
  'Bin7': '-1.20909,-0.88952',
  'Bin8': '-1.76935,-1.20909'},
 'Lib_1A': {'Bin1': '0.67715,0.96117',
  'Bin2': '0.36083,0.67715',
  'Bin3': '0.04782,0.36083',
  'Bin4': '-0.2673,0.04782',
  'Bin5': '-0.57833,-0.2673',
  'Bin6': '-0.89008,-0.57833',
  'Bin7': '-1.21064,-0.89008',
  'Bin8': '-1.8087,-1.21064'},
 'Lib_1B': {'Bin1': '0.67643,0.95528',
  'Bin2': '0.36088,0.67643',
  'Bin3': '0.04793,0.36088',
  'Bin4': '-0.26701,0.04793',
  'Bin5': '-0.57769,-0.26701',
  'Bin6': '-0.88959,-0.57769',
  'Bin7': '-1.21028,-0.88959',
  'Bin8': '-1.79785,-1.2102

In [7]:
cell_count = {}
for lib in libs:
    cell_count[lib] = {}
    for gate in gates:
        cell_count[lib][gate] = fcmeasure[lib+'_'+gate].data.count()[0]

In [8]:
cell_count_df = pd.DataFrame.from_dict(cell_count)
cell_count_df.to_csv('FACS_sorting_Data/cell_counts_20200423.csv')

In [9]:
bound_df = pd.DataFrame.from_dict(boundary)
bound_df.to_csv('FACS_sorting_Data/Boundary_determine20200303.csv')
Lib0_df = pd.DataFrame(columns=['Bin','R1','R2'])
Lib0_df['Bin'] = ['P2','P3','P4','P5','P6','P7','P8','P9']
Lib0_df['R1'] = bound_df['Lib_0A'].values
Lib0_df['R2'] = bound_df['Lib_0B'].values
Lib0_df.to_csv('FACS_sorting_Data/bin_boundary_configure_Lib0.csv',index='False')

Lib1_df = pd.DataFrame(columns=['Bin','R1','R2'])
Lib1_df['Bin'] = ['P2','P3','P4','P5','P6','P7','P8','P9']
Lib1_df['R1'] = bound_df['Lib_1A'].values
Lib1_df['R2'] = bound_df['Lib_1B'].values
Lib1_df.to_csv('FACS_sorting_Data/bin_boundary_configure_Lib1.csv',index='False')

Lib2_df = pd.DataFrame(columns=['Bin','R1','R2'])
Lib2_df['Bin'] = ['P2','P3','P4','P5','P6','P7','P8','P9']
Lib2_df['R1'] = bound_df['Lib_2A'].values
Lib2_df['R2'] = bound_df['Lib_2B'].values
Lib2_df.to_csv('FACS_sorting_Data/bin_boundary_configure_Lib2.csv',index='False')

Lib3_df = pd.DataFrame(columns=['Bin','R1','R2'])
Lib3_df['Bin'] = ['P2','P3','P4','P5','P6','P7','P8','P9']
Lib3_df['R1'] = bound_df['Lib_3A'].values
Lib3_df['R2'] = bound_df['Lib_3B'].values
Lib3_df.to_csv('FACS_sorting_Data/bin_boundary_configure_Lib3.csv',index='False')

Lib5_df = pd.DataFrame(columns=['Bin','R1','R2'])
Lib5_df['Bin'] = ['P2','P3','P4','P5','P6','P7','P8','P9']
Lib5_df['R1'] = bound_df['Lib_5A'].values
Lib5_df['R2'] = bound_df['Lib_5B'].values
Lib5_df.to_csv('FACS_sorting_Data/bin_boundary_configure_Lib5.csv',index='False')

Lib8_df = pd.DataFrame(columns=['Bin','R1','R2'])
Lib8_df['Bin'] = ['P2','P3','P4','P5','P6','P7','P8','P9']
Lib8_df['R1'] = bound_df['Lib_8A'].values
Lib8_df['R2'] = bound_df['Lib_8B'].values
Lib8_df.to_csv('FACS_sorting_Data/bin_boundary_configure_Lib8.csv',index='False')

In [10]:
cell_count_df

,Lib_0A,Lib_0B,Lib_1A,Lib_1B,Lib_2A,Lib_2B,Lib_3A,Lib_3B,Lib_5A,Lib_5B,Lib_8A,Lib_8B
P2,21997,22488,30548,26867,110301,101317,117575,115177,144317,124016,149162,129711
P3,54505,56355,76561,72069,246960,242859,247121,257105,265898,265093,268464,266789
P4,109894,112531,116113,114387,340727,339565,344205,338901,355529,348767,355153,348959
P5,116320,116065,120954,118033,350723,344477,351169,338767,354816,347612,358536,346521
P6,103793,101636,96247,93829,292187,287842,305561,302386,368718,346777,385172,361208
P7,92505,91540,116285,109563,413203,395977,430016,455553,535014,526005,550831,538571
P8,221377,237024,223305,229793,639627,642486,606635,640115,598428,634953,596091,632713
P9,308985,292475,249359,264918,659631,696320,614117,635716,567635,614238,571656,604089


In [11]:
bound_df = pd.DataFrame.from_dict(boundary)
bound_df.to_csv('FACS_sorting_Data/Boundary_determine20200423.csv')

In [12]:
bound_df

,Lib_0A,Lib_0B,Lib_1A,Lib_1B,Lib_2A,Lib_2B,Lib_3A,Lib_3B,Lib_5A,Lib_5B,Lib_8A,Lib_8B
Bin1,"0.67396,0.92539","0.67456,0.93268","0.67715,0.96117","0.67643,0.95528","0.67719,0.96591","0.678,0.96153","0.67781,0.96633","0.67791,0.96665","0.67487,0.9585","0.67561,0.95622","0.67373,0.95597","0.67577,0.95542"
Bin2,"0.36067,0.67396","0.36212,0.67456","0.36083,0.67715","0.36088,0.67643","0.3596,0.67719","0.35965,0.678","0.36016,0.67781","0.3595,0.67791","0.35706,0.67487","0.35682,0.67561","0.35631,0.67373","0.35758,0.67577"
Bin3,"0.04855,0.36067","0.04969,0.36212","0.04782,0.36083","0.04793,0.36088","0.04712,0.3596","0.04738,0.35965","0.04747,0.36016","0.0468,0.3595","0.04395,0.35706","0.04472,0.35682","0.04362,0.35631","0.04428,0.35758"
Bin4,"-0.26611,0.04855","-0.26546,0.04969","-0.2673,0.04782","-0.26701,0.04793","-0.26674,0.04712","-0.26663,0.04738","-0.26578,0.04747","-0.26619,0.0468","-0.26715,0.04395","-0.26714,0.04472","-0.26748,0.04362","-0.26682,0.04428"
Bin5,"-0.57916,-0.26611","-0.57726,-0.26546","-0.57833,-0.2673","-0.57769,-0.26701","-0.57705,-0.26674","-0.57743,-0.26663","-0.57684,-0.26578","-0.57709,-0.26619","-0.57855,-0.26715","-0.57836,-0.26714","-0.57893,-0.26748","-0.57841,-0.26682"
Bin6,"-0.89001,-0.57916","-0.88952,-0.57726","-0.89008,-0.57833","-0.88959,-0.57769","-0.89009,-0.57705","-0.88981,-0.57743","-0.89032,-0.57684","-0.89047,-0.57709","-0.89324,-0.57855","-0.89282,-0.57836","-0.89376,-0.57893","-0.89298,-0.57841"
Bin7,"-1.20973,-0.89001","-1.20909,-0.88952","-1.21064,-0.89008","-1.21028,-0.88959","-1.21112,-0.89009","-1.21071,-0.88981","-1.2102,-0.89032","-1.21146,-0.89047","-1.21387,-0.89324","-1.21301,-0.89282","-1.21438,-0.89376","-1.21321,-0.89298"
Bin8,"-1.78343,-1.20973","-1.76935,-1.20909","-1.8087,-1.21064","-1.79785,-1.21028","-1.84672,-1.21112","-1.86489,-1.21071","-1.85822,-1.2102","-1.86234,-1.21146","-1.89074,-1.21387","-1.88052,-1.21301","-1.89664,-1.21438","-1.88219,-1.21321"


In [13]:
from decimal import Decimal
a = 12.345
b = Decimal(a).quantize(Decimal("0.00"))

In [14]:
c= round(a, 5) 

In [15]:
cell_counts = cell_count_df.copy()
libs = ['0A','0B','1A','1B','2A','2B','3A','3B','5A','5B','8A','8B']
cerulenin = [0,1,2,3,5,8]
cells_ratio = pd.DataFrame(index = cell_counts.index)
for lib in libs:
    cells_ratio['Lib_'+lib] = cell_counts[['Lib_'+lib]].apply(lambda x: x/(np.sum(x)))

In [16]:
cells_ratio

,Lib_0A,Lib_0B,Lib_1A,Lib_1B,Lib_2A,Lib_2B,Lib_3A,Lib_3B,Lib_5A,Lib_5B,Lib_8A,Lib_8B
P2,0.021369,0.021831,0.029676,0.026098,0.036124,0.033210,0.038979,0.037350,0.045235,0.038665,0.046108,0.040176
P3,0.052950,0.054708,0.074376,0.070007,0.080881,0.079604,0.081926,0.083375,0.083344,0.082649,0.082986,0.082634
P4,0.106758,0.109241,0.112800,0.111114,0.111591,0.111302,0.114111,0.109900,0.111439,0.108736,0.109782,0.108085
P5,0.113000,0.112672,0.117503,0.114655,0.114865,0.112912,0.116420,0.109857,0.111215,0.108376,0.110828,0.107330
P6,0.100831,0.098665,0.093501,0.091144,0.095694,0.094348,0.101300,0.098059,0.115573,0.108116,0.119062,0.111879
P7,0.089865,0.088864,0.112967,0.106428,0.135327,0.129793,0.142559,0.147728,0.167697,0.163994,0.170269,0.166815
P8,0.215059,0.230095,0.216933,0.223217,0.209483,0.210593,0.201112,0.207579,0.187574,0.197961,0.184259,0.195974
P9,0.300167,0.283925,0.242244,0.257337,0.216035,0.228239,0.203593,0.206152,0.177922,0.191503,0.176706,0.187108


In [17]:
cells_ratio.to_excel('FACS_sorting_Data/cell_ratio_0423.xlsx')

In [18]:
float(boundary['Lib_0A']['Bin1'].split(',')[0])

0.67396

In [19]:
ratios = {}
libs = ['Lib_0A','Lib_0B','Lib_1A','Lib_1B','Lib_2A','Lib_2B',\
        'Lib_3A','Lib_3B','Lib_5A','Lib_5B','Lib_8A','Lib_8B']
for lib in libs:
    ratios[lib] = {}
    print(lib)
    ratios[lib]['Bin1'] = round((P2P3[lib]+P2_95[lib])/2,5)
    ratios[lib]['Bin2'] = round((P3P4[lib]+P2P3[lib])/2,5)
    ratios[lib]['Bin3'] = round((P4P5[lib]+P3P4[lib])/2,5)
    ratios[lib]['Bin4'] = round((P5P6[lib]+P4P5[lib])/2,5)
    ratios[lib]['Bin5'] = round((P6P7[lib]+P5P6[lib])/2,5)
    ratios[lib]['Bin6'] = round((P7P8[lib]+P6P7[lib])/2,5)
    ratios[lib]['Bin7'] = round((P8P9[lib]+P7P8[lib])/2,5)
    ratios[lib]['Bin8'] = round((P9_5[lib]+P8P9[lib])/2,5)

Lib_0A
Lib_0B
Lib_1A
Lib_1B
Lib_2A
Lib_2B
Lib_3A
Lib_3B
Lib_5A
Lib_5B
Lib_8A
Lib_8B


In [20]:
ratios_df = pd.DataFrame.from_dict(ratios)

In [21]:
ratios_df.to_excel('FACS_sorting_Data/ratios_0423.xlsx')